In [1]:
import numpy
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
import mysql.connector
from mysql.connector import errorcode

init parameter

##load image databases, and information


In [2]:
sqlContext=SQLContext(sc)
hiveContext=HiveContext(sc)

hiveContext.sql('use aibgraz')

DataFrame[]

In [4]:
df_graz02 = sqlContext.read.format('jdbc').options(
    url='jdbc:mysql://127.0.0.1:3306',
    dbtable='imgBase.graz_02',
    driver='com.mysql.jdbc.Driver',
    user='sh').load()

df_graz02_info = sqlContext.read.format('jdbc').options(
    url='jdbc:mysql://127.0.0.1:3306',
    dbtable='imgBase.graz_02_info',
    driver='com.mysql.jdbc.Driver',
    user='sh').load()
#type(df_graz02)
#print(df_graz02.count())

rdd_graz02 = df_graz02.rdd
hdf_graz02=hiveContext.createDataFrame(rdd_graz02)
#hdf_graz02.write()

#df_graz02.write.format("jdbc").options(url='jdbc:hive2://localhost:10000',
#                                       dbtable='aibgraz.graz_02',
#                                       user='sh').save()
            

In [4]:
hiveContext.sql('use aibgraz')

hiveContext.sql('drop table  IF EXISTS graz02')
hdf_graz02.write.saveAsTable(name='graz02')

In [6]:
df_graz02_info.show()
hdf_graz02_info=hiveContext.createDataFrame(df_graz02_info.rdd)
hdf_graz02_info.write.saveAsTable(name='graz02_cate_info')

hiveContext.sql('select * from graz02_cate_info').show()
hiveContext.sql('show table extended like graz02_cate_info').show()

+--------+---+------------+----+---------------+----------+
|category|num|isbackground| ext|groundtruth_ext|img_prefix|
+--------+---+------------+----+---------------+----------+
|    bike|365|       false|.bmp|           .jpg|     bike_|
|    cars|420|       false|.bmp|           .jpg|     cars_|
|    none|380|        true|.bmp|           .jpg|  bk_graz_|
|  person|311|       false|.bmp|           .jpg|   person_|
+--------+---+------------+----+---------------+----------+



+--------+---+------------+----+---------------+----------+
|category|num|isbackground| ext|groundtruth_ext|img_prefix|
+--------+---+------------+----+---------------+----------+
|    bike|365|       false|.bmp|           .jpg|     bike_|
|    cars|420|       false|.bmp|           .jpg|     cars_|
|    none|380|        true|.bmp|           .jpg|  bk_graz_|
|  person|311|       false|.bmp|           .jpg|   person_|
+--------+---+------------+----+---------------+----------+

+--------------------+
|              result|
+--------------------+
|tableName:graz02_...|
|            owner:sh|
|location:hdfs://l...|
|inputformat:org.a...|
|outputformat:org....|
|columns:struct co...|
|   partitioned:false|
|   partitionColumns:|
|  totalNumberFiles:4|
|  totalFileSize:3475|
|    maxFileSize:1492|
|       minFileSize:0|
|lastAccessTime:14...|
|lastUpdateTime:14...|
|                    |
+--------------------+



In [3]:
hiveContext.sql('show table extended like graz02').show()
hiveContext.sql('select count(*) from graz02').show()

+--------------------+
|              result|
+--------------------+
|    tableName:graz02|
|            owner:sh|
|location:hdfs://l...|
|inputformat:org.a...|
|outputformat:org....|
|columns:struct co...|
|   partitioned:false|
|   partitionColumns:|
|  totalNumberFiles:4|
| totalFileSize:19326|
|   maxFileSize:17193|
|       minFileSize:0|
|lastAccessTime:14...|
|lastUpdateTime:14...|
|                    |
+--------------------+

+--------+
|count(1)|
+--------+
|    1476|
+--------+



bag of feature image categorization

select category/training/testing

In [4]:
def sel(r):
    if (r.category=='person') and (r.cat_ind<=10):
        return True
    else:
        return False
    
rdd_graz02_select = rdd_graz02.filter(sel);
rdd_graz02_select.first()

Row(ind=1166, cat_ind=1, name=u'person_001', category=u'person', path=u'/home/sh/Data/TestData/Graz-02/data/person/person_001.bmp', groundtruth_path=u'/home/sh/Data/TestData/Graz-02/groundtruth/person/person_001_gt.jpg')


##feature extraction


In [9]:
# load image from Row.path,
# extract feature of that image
# and save into file, whose path is Row.

import cv2
from scipy.misc import imread


def block_feat(Row):
    img = imread(Row.path)
    featureDetector=cv2.xfeatures2d.SIFT_create()
    descExtractor = featureDetector
    
    kp = featureDetector.detect(img, None)
    _kp,des = descExtractor.compute(img, kp)
    return des


rdd_feat = rdd_graz02_select.map(block_feat)
print(rdd_feat.count())
print(type(rdd_feat.first()))
#print(rdd_feat.first())

10
<type 'numpy.ndarray'>


In [10]:
print(rdd_feat.first().shape)

(488, 128)


##train dictionary
kmeans

In [ ]:
def block_dictionary():
    
dic_ = rdd_feat.reduce(block_dictionary)